# Предсказание стоимости квартир в Нижнем Новгороде

Запуск блокнота предполагается как на локальном компьютере, так и в Colab. Поэтому ниже реализован алгоритм определения где был запущен блокнот.

Также отметим, что при запуске в Colab необходимо предоставить доступ к google drive.

In [1]:
try:
  from google.colab import files
  from google.colab import drive

  drive.mount('/content/drive')

  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
import pandas as pd
import re

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [4]:
!pip install -U phik --quiet

In [5]:
from phik import resources, report
import matplotlib.pyplot as plt

import seaborn as sns

In [16]:
!pip install -U pandas-profiling --quiet

In [17]:
#from pandas_profiling import ProfileReport
from ydata_profiling import ProfileReport

In [18]:
!pip install -U catboost --quiet
from catboost import CatBoostRegressor

In [19]:
from datetime import datetime

Чтение данных

In [20]:
path_data = ''
if IN_COLAB:
  path_data = '/content/drive/MyDrive/Colab Notebooks/apartment-price-prediction/'

data = pd.read_csv(path_data + 'dataset/data_flats.csv', decimal=',')

In [21]:
data.head()

,Unnamed: 0,Наименование,Тип жилья,Общая площадь,Жилая площадь,Площадь кухни,Высота потолков,Санузел,Ремонт,Год постройки,...,Аварийность,Этаж,Адрес,Цена,Балкон/лоджия,Вид из окон,Количество лифтов,Парковка,Газоснабжение,Строительная серия
0,0,2-комн. квартира,Вторичка,60.0,28.0,18.0,"2,8 м",1 раздельный,Евроремонт,2017.0,...,Нет,26 из 28,"Нижегородская область, Нижний Новгород, р-н Со...",9 000 000 ₽,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3-комн. квартира,Вторичка,104.0,49.7,22.1,"2,7 м",2 совмещенных,Без ремонта,2023.0,...,Нет,6 из 9,"Нижегородская область, Нижний Новгород, р-н Со...",18 800 000 ₽,1 лоджия,На улицу и двор,1 грузовой,Подземная,NaN,NaN
2,2,1-комн. квартира,Вторичка,50.0,16.7,11.7,"2,7 м",1 совмещенный,Без ремонта,2023.0,...,Нет,6 из 10,"Нижегородская область, Нижний Новгород, р-н Со...",8 150 000 ₽,1 лоджия,Во двор,1 пассажирский,Наземная,NaN,NaN
3,3,3-комн. квартира,Вторичка,56.0,39.0,7.0,"2,5 м",1 раздельный,Евроремонт,1972.0,...,Нет,7 из 9,"Нижегородская область, Нижний Новгород, р-н Ав...",6 300 000 ₽,1 лоджия,На улицу,1 пассажирский,Наземная,Центральное,NaN
4,4,3-комн. квартира,Вторичка,60.2,44.2,7.0,"2,65 м",1 раздельный,Евроремонт,1917.0,...,Нет,1 из 9,"Нижегородская область, Нижний Новгород, р-н Пр...",6 990 000 ₽,1 лоджия,На улицу и двор,1 пассажирский,NaN,Центральное,NaN


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508 entries, 0 to 1507
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          1508 non-null   int64  
 1   Наименование        1508 non-null   object 
 2   Тип жилья           1508 non-null   object 
 3   Общая площадь       1508 non-null   float64
 4   Жилая площадь       1459 non-null   float64
 5   Площадь кухни       1498 non-null   float64
 6   Высота потолков     487 non-null    object 
 7   Санузел             1320 non-null   object 
 8   Ремонт              1366 non-null   object 
 9   Год постройки       1464 non-null   float64
 10  Мусоропровод        427 non-null    object 
 11  Тип дома            1231 non-null   object 
 12  Тип перекрытий      1449 non-null   object 
 13  Подъезды            1424 non-null   float64
 14  Отопление           1449 non-null   object 
 15  Аварийность         1449 non-null   object 
 16  Этаж  

In [ ]:
pd.DataFrame(round(data.isna().mean()*100,1)).style.background_gradient('coolwarm')

In [ ]:
data = data.drop([
    'Unnamed: 0',
    'Мусоропровод',
    'Вид из окон',
    'Парковка',
    'Строительная серия',
    'Высота потолков'
], axis=1)

In [ ]:
# Для быстрого перехода к обучению исключин вначале столбцы с большими пропусками
# data = data.drop([
#    'Тип дома'
#], axis=1)

In [ ]:
data.duplicated().sum()

In [ ]:
len(data)

Выводы по обзору данных:
- Требуется предобработка;
- Необходимо переименование столбцов;
- Требуется извлечение признаков из данных;
- Требуется изменение типов данных.

## Предобработка данных

Переименуем столбцы

In [ ]:
data = (data.rename(
    columns={
        'Наименование': 'name',
        'Тип жилья': 'type_of_housing',
        'Общая площадь': 'total_area',
        'Жилая площадь': 'living_area',
        'Площадь кухни': 'kitchen_area',
        'Высота потолков': 'ceiling_height',
        'Этаж': 'floor',
        'Санузел': 'bathroom',
        'Балкон/лоджия': 'balcony/logie',
        'Вид из окон': 'view_from_the_window',
        'Ремонт': 'renovation',
        'Год постройки': 'year',
        'Строительная серия': 'construction_series',
        'Мусоропровод': 'garbage_chute',
        'Количество лифтов': 'type_of_elevators',
        'Тип перекрытий': 'floor_type',
        'Парковка': 'parking',
        'Подъезды': 'entrances',
        'Отопление': 'heating',
        'Аварийность': 'accident_rate',
        'Газоснабжение': 'gas_supply',
        'Цена': 'price',
        'Тип дома': 'type_of_building',
        'Адрес': 'address'
    })
)

### Обработка поля наименование жилья

In [ ]:
data['name'].unique()

In [ ]:
data['name'].value_counts()

In [ ]:
def get_rooms(input_string):
    for rooms in range(1, 5):
        if str(rooms) in input_string:
            return rooms
    else:
        if 'студия' in input_string.lower():
            return 1
    return None

In [ ]:
print(get_rooms('1-комн. квартира'))
print(get_rooms('5-комн. квартира'))
print(get_rooms('0-комн. квартира'))
print(get_rooms('4-комн. квартира'))

In [ ]:
def is_apartments(input_string):
    if 'апартаменты' in input_string.lower():
        return 1
    else:
        return 0

In [ ]:
print(is_apartments('3-комн. квартира'))
print(is_apartments('Студия'))
print(is_apartments('1-комн. апартаменты'))
print(is_apartments('Апартаменты-студия'))

In [ ]:
def is_studio(input_string):
    if 'студия' in input_string.lower():
        return 1
    else:
        return 0

In [ ]:
print(is_studio('3-комн. квартира'))
print(is_studio('Студия'))
print(is_studio('1-комн. апартаменты'))
print(is_studio('Апартаменты-студия'))

In [ ]:
data['rooms'] = data['name'].apply(get_rooms)

In [ ]:
data['rooms'].unique()

In [ ]:
data['apartments'] = data['name'].apply(is_apartments)

In [ ]:
data['apartments'].unique()

In [ ]:
data['studio'] = data['name'].apply(is_studio)

In [ ]:
data['studio'].unique()

In [ ]:
data = data.drop('name', axis=1)

In [ ]:
data = data.dropna(subset=['rooms'])

In [ ]:
data['rooms'] = data['rooms'].astype('int')

### Обработка поля тип жилья

In [ ]:
data['type_of_housing'].value_counts()

In [ ]:
data = data.drop('type_of_housing', axis=1)

In [ ]:
data['total_area'].min()

In [ ]:
data['total_area'].max()

In [ ]:
data[data['living_area'].isna()].sample(2)

In [ ]:
list_rooms = data.loc[data['living_area'].isna(), 'rooms'].unique()
for rooms in list_rooms:
    coeff = (
        data.loc[data['rooms'] == rooms, 'living_area'].mean() /
        data.loc[data['rooms'] == rooms, 'total_area'].mean()
    )
    data.loc[(data['rooms'] == rooms) & (data['living_area'].isna()), 'living_area'] =\
    round(data.loc[(data['rooms'] == rooms) & (data['living_area'].isna()), 'total_area'] * coeff, 1)

In [ ]:
data['living_area'].min()

In [ ]:
data['living_area'].max()

In [ ]:
data[data['kitchen_area'].isna()].sample(2)

In [ ]:
list_rooms = data.loc[data['kitchen_area'].isna(), 'rooms'].unique()
for rooms in list_rooms:
    coeff = (
        data.loc[data['rooms'] == rooms, 'kitchen_area'].mean() /
        data.loc[data['rooms'] == rooms, 'total_area'].mean()
    )
    data.loc[(data['rooms'] == rooms) & (data['kitchen_area'].isna()), 'kitchen_area'] =\
    round(data.loc[(data['rooms'] == rooms) & (data['kitchen_area'].isna()), 'total_area'] * coeff, 1)

In [ ]:
data['kitchen_area'].min()

In [ ]:
data['kitchen_area'].max()

In [ ]:
# data['ceiling_height'].unique()

In [ ]:
# data['ceiling_height'] = data['ceiling_height'].str.extract(r'(\d\,\d|\d+)')
# data['ceiling_height'] = data['ceiling_height'].str.replace(',', '.')

In [ ]:
# data['ceiling_height'].unique()

In [ ]:
# data['ceiling_height'] = pd.to_numeric(data['ceiling_height'])

In [ ]:
# data['ceiling_height'].min()

In [ ]:
# data['ceiling_height'].max()

In [ ]:
data['bathroom'] = data['bathroom'].fillna(value='unknow')

In [ ]:
data['bathroom'].value_counts()

In [ ]:
def extract_number_of_bathrooms(input_string):
    string_list = re.findall(r'\d+', input_string)
    int_list = list(map(int, string_list))
    number_of_bathrooms = sum(int_list)
    if number_of_bathrooms == 0:
        return 1

    return number_of_bathrooms

In [ ]:
extract_number_of_bathrooms(' unknow')

In [ ]:
extract_number_of_bathrooms(' 5 unknow, 1')

In [ ]:
data['number_of_bathrooms'] = data['bathroom'].apply(extract_number_of_bathrooms)

In [ ]:
data = data.drop('bathroom', axis=1)

In [ ]:
data['gas_supply'].unique()

In [ ]:
data['gas_supply'] = data['gas_supply'].fillna(value='Нет информации')

In [ ]:
data['gas_supply'].value_counts()

In [ ]:
data.loc[data['gas_supply'] == 'Центральное', 'year'].max()

Выведем данные по недвижимости имеющие автономное газоснабжение

In [ ]:
data[data['gas_supply'] == 'Автономное']

Рассмотрению подлежит недвижимость - квартиры, имеющие централизированное газоснабжение. Поэтому исключим недвижимость с автономным газоснабжением

In [ ]:
data = data[data['gas_supply'] != 'Автономное']

In [ ]:
data[(data['gas_supply'] == 'Центральное') & (data['year'] == 2023)]

### Обработка поля ремонт

In [ ]:
data['renovation'].unique()

In [ ]:
data['renovation'] = data['renovation'].fillna(value='unknow')

In [ ]:
data['renovation'].value_counts()

### Обработка поля год постройки

In [ ]:
data['year'].unique()

In [ ]:
data['year'].isna().sum()

Год постройки это категориальный параметр. Поэтому из года постройки лучше получить его числовой парамтер - возраст.

Исключаем квартиры с неизвестным годом постройки.

In [ ]:
data = data.dropna(subset=['year'])

In [ ]:
data.loc[:, 'year'] = pd.to_numeric(data['year'])
data.loc[:, 'year'] = data['year'].astype('int')

In [ ]:
current_year = datetime.now().year
data['house_age'] = current_year - data['year']

In [ ]:
data = data.drop('year', axis=1)

In [ ]:
data['balcony/logie'].unique()

In [ ]:
data['balcony/logie'] = data['balcony/logie'].fillna(value='unknow')

In [ ]:
data['balcony/logie'].value_counts()

In [ ]:
def is_balcony(input_string):
    if 'балк' in input_string.lower():
        return 1
    else:
        return 0

In [ ]:
def is_logie(input_string):
    if 'лодж' in input_string.lower():
        return 1
    else:
        return 0

In [ ]:
data['balcony'] = data['balcony/logie'].apply(is_balcony)

In [ ]:
data['logie'] = data['balcony/logie'].apply(is_logie)

In [ ]:
data = data.drop('balcony/logie', axis=1)

In [ ]:
# data['view_from_the_window'].unique()

In [ ]:
# data['view_from_the_window'] = data['view_from_the_window'].fillna(value='unknow')

In [ ]:
# data['view_from_the_window'].value_counts()

In [ ]:
data['entrances'].unique()

In [ ]:
data['entrances'] = data['entrances'].fillna(value='unknow')

In [ ]:
data['entrances'].value_counts()

In [ ]:
data['heating'].unique()

In [ ]:
data['heating'] = data['heating'].fillna(value='Нет информации')

In [ ]:
data['heating'].value_counts()

In [ ]:
data['accident_rate'].unique()

In [ ]:
data['accident_rate'] = data['accident_rate'].fillna(value='Нет информации')

In [ ]:
data['accident_rate'].value_counts()

In [ ]:
data = data[data['accident_rate'] != 'Да']

In [ ]:
data['floor'].sample(2)

In [ ]:
data.loc[:, 'floors_total'] = data['floor'].apply(lambda x: re.findall(r'(\d+)', x)[1])

In [ ]:
data['floors_total'].info()

In [ ]:
data['floors_total'] = pd.to_numeric(data['floors_total'])

In [ ]:
data['floors_total'].min()

In [ ]:
data['floors_total'].value_counts()

In [ ]:
data[data['floors_total'] < 5].sample(5)

In [ ]:
data = data[data['floors_total'] >= 5 ]

In [ ]:
data['floors_total'].max()

In [ ]:
data.loc[:, 'floor'] = data['floor'].apply(lambda x: re.findall(r'(\d+)', x)[0])

In [ ]:
data['floor'].info()

In [ ]:
data['floor'] = pd.to_numeric(data['floor'])

In [ ]:
data['floor'].value_counts()

In [ ]:
data.loc[data['floors_total'] < data['floor'], 'floor'].count()

In [ ]:
data['first_floor'] = data['floor'].apply(lambda x: 1 if (x == 1) else 0)
data['last_floor'] = (data['floor'] == data['floors_total']).apply(lambda x: 1 if x else 0)

In [ ]:
# data = data.drop(['floor', 'floors_total'], axis=1)

In [ ]:
data['floor_type'].unique()

In [ ]:
data['floor_type'] = data['floor_type'].fillna('Нет информации')

In [ ]:
data['floor_type'].value_counts()

In [ ]:
data[data['floor_type'] == 'Деревянные']

In [ ]:
data = data[(data['floor_type'] != 'Деревянные') & (data['floor_type'] != 'Смешанные') ]

In [ ]:
print(data.loc[0, 'address'])

In [ ]:
data['region'] = data['address'].apply(lambda x: x.split(',')[2])

In [ ]:
list_region = data['region'].unique()
list_region

In [ ]:
list_region_new = ['Советский', 'Автозаводский', 'Приокский',
       'Нижегородский', 'Ленинский', 'Канавинский',
       'Московский', 'Сормовский', 'Кудьма поселок']

In [ ]:
data['region'] = data['region'].replace(list_region, list_region_new)

In [ ]:
data['region'].value_counts()

In [ ]:
data = data[data['region'] != 'Кудьма поселок']

In [ ]:
data = data.drop('address', axis=1)

In [ ]:
data['price'].sample(5)

In [ ]:
data['price'] = data['price'].str.findall(r'(\d\,.\d|\d+)').str.join('')

In [ ]:
data['price'] = pd.to_numeric(data['price'])

In [ ]:
data['price'].min()

In [ ]:
data['price'].max()

In [ ]:
data['price'].hist(bins=50)

In [ ]:
data = data[data['price'] < 10e6]

### Обработка столбца - количество лифтов

In [ ]:
data['type_of_elevators'].isna().sum()

In [ ]:
data['type_of_elevators'] = data['type_of_elevators'].fillna('Нет информации')

In [ ]:
data['type_of_elevators'].value_counts()

In [ ]:
data.loc[data['type_of_elevators'] == 'Нет информации', 'floors_total'].value_counts()

In [ ]:
data.loc[data['floors_total'] == 5, 'type_of_elevators'].value_counts()

In [ ]:
# data.loc[data['type_of_elevators'] != 'Нет информации', 'elevator'] = 1

In [ ]:
# data.loc[data['type_of_elevators'] == 'Нет информации', 'elevator'] = 0

In [ ]:
def is_service_elevator(input_string):
    if 'груз' in input_string.lower():
        return 1
    else:
        return 0

In [ ]:
def is_passenger_elevator(input_string):
    if 'пассажир' in input_string.lower():
        return 1
    else:
        return 0

In [ ]:
data['service_elevator'] = data['type_of_elevators'].apply(is_service_elevator)

In [ ]:
data['passenger_elevator'] = data['type_of_elevators'].apply(is_passenger_elevator)

In [ ]:
data = data.drop('type_of_elevators', axis=1)

In [ ]:
pd.DataFrame(round(data.isna().mean()*100,1)).style.background_gradient('coolwarm')

In [ ]:
data.info()

- [ ] Наименование
- [ ] Тип жилья
- [ ] Общая площадь
- [ ] Жилая площадь
- [ ] Площадь кухни
- [ ] Высота потолков
- [ ] Санузел
- [x] Ремонт
- [ ] Год постройки
- [ ] Мусоропровод
- [ ] Тип дома
- [ ] Тип перекрытий
- [ ] Подъезды
- [ ] Отопление
- [ ] Аварийность
- [ ] Этаж
- [ ] Адрес
- [ ] Цена
- [ ] Балкон/лоджия
- [ ] Вид из окон
- [ ] Количество лифтов
- [ ] Парковка
- [ ] Газоснабжение
- [ ] Строительная серия

In [ ]:
# data = data.drop(['entrances', 'year'], axis=1)

In [ ]:
#data.loc[:, 'year'] = data['year'].astype(str)

In [ ]:
data.loc[:, 'entrances'] = data['entrances'].astype(str)

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)

### Обработка признака тип дома

In [ ]:
data['type_of_building'].unique()

In [ ]:
data['type_of_building'] = data['type_of_building'].fillna(value="Нет информации")

In [ ]:
data['type_of_building'].value_counts()

## Исследовательский анализ данных

In [ ]:
data.sample(2)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.suptitle('Распределение квартир по количеству комнат', fontsize=11, fontweight='bold', y=0.95)
sns.histplot(data=data, y='rooms')
ax.set(xlabel='Количество квартир, шт.', ylabel='Количество комнат')
plt.show();

Из графика видно, что однокомнатные квартиры имеют большее распространение.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.suptitle('Распределение квартир по районам', fontsize=11, fontweight='bold', y=0.95)
sns.histplot(data=data, y='region')
ax.set(xlabel='Количество квартир, шт.', ylabel='Район квартиры')
plt.show();

Наибольшее количество квартир в данных имеют Автозаводский район. Меньшее количество квартир представленно Нижегородским районом.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.suptitle('Распределение квартир по площадям', fontsize=11, fontweight='bold', y=0.95)
sns.histplot(data=data, x='total_area')
ax.set(xlabel='Площадь квартиры', ylabel='Количество квартир, шт.')
plt.show();

Среднее значение общей площади квартир представленных в выборке находится около 40 кв.м.

In [ ]:
fig = plt.figure(figsize=[10, 5])
ax = fig.add_subplot()
fig.suptitle('Распределение квартир по районам', fontsize=11, fontweight='bold', y=0.95)
sns.countplot(data=data, x='region', hue='rooms')
ax.set(xlabel='Район', ylabel='Количество')
ax.legend(title="Количество комнат")
plt.xticks(rotation = 45)
plt.show();

In [ ]:
fig = plt.figure(figsize=[10, 5])
ax = fig.add_subplot()
fig.suptitle('Распределение стоимости квартир по районам', fontsize=11, fontweight='bold', y=0.95)
sns.barplot(data=data, x='region', y='price', hue='rooms')
ax.set(xlabel='Район', ylabel='Стоимость')
ax.legend(title="Количество комнат")
plt.xticks(rotation = 45)
plt.show();

In [ ]:
price_for_sq_metr = (
    data.groupby('rooms')['price'].mean()
    /
    data.groupby('rooms')['total_area'].mean()
)


fig = plt.figure()
ax = fig.add_subplot()
fig.suptitle('Распределение стоимости квадратного метра по типу квартиры', fontsize=11, fontweight='bold', y=0.95)
sns.barplot(x=price_for_sq_metr.index, y=price_for_sq_metr)
ax.set(xlabel='Тип квартиры', ylabel='Стоимость кв. м., тыс. руб.')
plt.xticks(rotation = 45)
plt.show();

In [ ]:
sns.scatterplot(data=data, x="total_area", y="price")

In [ ]:
sns.scatterplot(data=data, x="living_area", y="price")

In [ ]:
data.info()

### Генерация признаков

In [ ]:
data = data.drop('heating', axis=1)

In [ ]:
data['ref_area'] = data['living_area'] / data['rooms']

In [ ]:
data['ref_liv_kitch'] = data['living_area'] / data['kitchen_area']

**Корреляция признаков:**

In [ ]:
sns.set(font_scale=0.7)
plt.figure(figsize=(10, 10));
sns.heatmap(
    data.phik_matrix(),
    cmap='RdYlGn',
    annot=True,
    vmin=0, vmax=1);

apartments, floor_type

По корреляционной матрицы можно сделать вывод, что на целевой признак (price) минимальное значение оказывает признак этаж квартиры (floor). Возможно это связано с тем, что квартир располагающихся на первом этаже большинство в представленных данных. Таже малое влияние оказывает признак жилая площадь (living_area).

In [ ]:
data = data.drop(['apartments', 'accident_rate', 'floor_type', 'living_area',
                  'entrances', 'first_floor', 'last_floor', 'kitchen_area'], axis=1)

In [ ]:
sns.set(font_scale=0.7)
plt.figure(figsize=(10, 10));
sns.heatmap(
    data.phik_matrix(),
    cmap='RdYlGn',
    annot=True,
    vmin=0, vmax=1);

## Подготовка тренировочной и тестовой выборок

In [ ]:
RANDOM_STATE = 221023

Разделим данные на признаки и целевой признак

In [ ]:
features = data.drop(['price'], axis=1)
target= data['price']

In [ ]:
features.duplicated().sum()

In [ ]:
duplicate_indexes = features.duplicated()

In [ ]:
features = features[~duplicate_indexes]
target = target[~duplicate_indexes]

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM_STATE)

Посмотрим на размеры получившихся выборок

In [ ]:
print(features_train.shape)
print(target_train.shape)

print(features_test.shape)
print(target_test.shape)

Для категориальных признаков выполнил прямое кодирование, а числовые признаки промасштабируем.

In [ ]:
cat_cols = ['renovation', 'region', 'gas_supply', 'type_of_building']

num_cols = list(features_train.drop(cat_cols, axis=1).columns)
num_cols

In [ ]:
data[cat_cols].sample(5)

In [ ]:
cat_renovation = data['renovation'].unique()
cat_region = data['region'].unique()
cat_gas_supply = data['gas_supply'].unique()
cat_type_of_building = data['type_of_building'].unique()

encoder_cat = OneHotEncoder(
    categories=[cat_renovation, cat_region, cat_gas_supply, cat_type_of_building],
    handle_unknown='ignore'
)

In [ ]:
scaler = StandardScaler()

In [ ]:
transformer = ColumnTransformer([('cat_cols', encoder_cat, cat_cols),
                                ('num_cols', scaler, num_cols)])

Создадим функцию поиска оптимальных гиперпараметров моделей

In [ ]:
def make_grid_search(model, param_grid, features_train, target_train):
    pipe = Pipeline([('preprocessing', transformer), ('model', model)])
    grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=5, scoring='neg_root_mean_squared_error', verbose=2)

    model_grid = grid_search.fit(features_train, target_train)

    print(f"Лучшие параметры модели:")
    print(grid_search.best_params_)
    print('-'*10)
    print(f"Оценка RMSE модели: {grid_search.best_score_:.3f}")

    return model_grid

## Обучение моделей

### Обучение линейной регрессии

In [ ]:
%%time
model = LinearRegression()
param_grid={}

model_lr = make_grid_search(model, param_grid, features_train, target_train);

In [ ]:
predictions_test = model_lr.predict(features_test)

result = mean_squared_error(target_test, predictions_test)**0.5
print(result)

In [ ]:
%%time
model = LinearRegression()
param_grid={}

model_lr = make_grid_search(model, param_grid, features, target);

### Обучение дерева решений

In [ ]:
%%time
model = DecisionTreeRegressor(random_state=RANDOM_STATE)
param_grid={
             'model__max_depth':list(range(1, 15, 4))
           }

model_dtr = make_grid_search(model, param_grid, features_train, target_train)

In [ ]:
predictions_test = model_dtr.predict(features_test)

result = mean_squared_error(target_test, predictions_test)**0.5
print(result)

### Обучение модели градиентного бустинга CatBoostRegressor

In [ ]:
%%time
model = CatBoostRegressor(loss_function="RMSE", iterations=100, random_state=RANDOM_STATE, verbose=0)
param_grid={
              'model__depth':list(range(1, 9, 2)),
              'model__learning_rate':list([x/10 for x in range(1, 7, 2)])
           }

model_cbr = make_grid_search(model, param_grid, features_train, target_train)

In [ ]:
from catboost import CatBoostClassifier, Pool, metrics, cv

In [ ]:
model2 = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent'
)

In [ ]:
#cat_features
model = CatBoostRegressor(loss_function="RMSE", iterations=100, random_state=RANDOM_STATE, verbose=0)

## Тестирование

In [ ]:
predictions_test = model_cbr.predict(features_test)

result = mean_squared_error(target_test, predictions_test)**0.5
print(result)

Модель CatBoostRegressor показала значение метрики RMSE равным 645481.

In [ ]:
flat_me = [{
    'total_area': 63,
    'house_age': 2024-1985,
    'floors_total': 9,
    'rooms': 3,
    'gas_supply': 'Центральное',
    'passenger_elevator': 1,
    'region': 'Приокский',
    'ref_liv_kitch': 3.6,
    'ref_area': 11,
    'renovation': 'Евроремонт',
    'floor': 7,
    'type_of_building': 'Панельный',
    'logie': 1,
    'balcony': 1,
    'number_of_bathrooms': 1,
    'studio': 0,
    'service_elevator': 0
}]

In [ ]:
flat_sister = [{
    'total_area': 43,
    'house_age': 2024-1980,
    'floors_total': 5,
    'rooms': 2,
    'gas_supply': 'Центральное',
    'passenger_elevator': 0,
    'region': 'Автозаводский',
    'ref_liv_kitch': 4,
    'ref_area': 12,
    'renovation': 'Евроремонт',
    'floor': 2,
    'type_of_building': 'Панельный',
    'logie': 0,
    'balcony': 1,
    'number_of_bathrooms': 1,
    'studio': 0,
    'service_elevator': 0
}]

In [ ]:
(12+12)/6

In [ ]:
(12+12)/2

In [ ]:
data_flat_me = pd.DataFrame(flat_me)
data_flat_sister = pd.DataFrame(flat_sister)

In [ ]:
data_flat_me.head()

In [ ]:
data_flat_sister.head()

In [ ]:
round(model_cbr.predict(data_flat_me)[0])

In [ ]:
round(model_cbr.predict(data_flat_sister)[0])

Обучим модель на полном наборе данных - тренировочная выборка плюс тестовая выборка:

In [ ]:
%%time
model = CatBoostRegressor(loss_function="RMSE", iterations=100, random_state=RANDOM_STATE, verbose=0)
param_grid={
              'model__depth':list(range(1, 9, 2)),
              'model__learning_rate':list([x/10 for x in range(1, 7, 2)])
           }

model_cbr = make_grid_search(model, param_grid, features, target)

Проанализируем модель

In [ ]:
data_predict = data.copy()
data_predict['predict_error'] = (
    100 * (model_cbr.predict(features) - target) / target
)
data_predict['predict_error_abs'] = abs(
    100 * (model_cbr.predict(features) - target) / target
)

In [ ]:
fig = plt.figure(figsize=[10, 5])
ax = fig.add_subplot()
fig.suptitle('Распределение ошибки предсказания цены квартиры по районам', fontsize=11, fontweight='bold', y=0.95)
sns.barplot(data=data_predict, x='region', y='predict_error', hue='rooms')
ax.set(xlabel='Район', ylabel='Ошибка, %')
ax.legend(title="Количество комнат")
plt.xticks(rotation = 45)
plt.show();

In [ ]:
fig = plt.figure(figsize=[10, 5])
ax = fig.add_subplot()
fig.suptitle('Распределение ошибки предсказания цены квартиры по районам', fontsize=11, fontweight='bold', y=0.95)
sns.barplot(data=data_predict, x='region', y='predict_error_abs', hue='rooms')
ax.set(xlabel='Район', ylabel='Ошибка, %')
ax.legend(title="Количество комнат")
plt.xticks(rotation = 45)
plt.show();

Посмотрим на значимость признаков для модели.

In [ ]:
feature_importance = model_cbr.best_estimator_.named_steps["model"].feature_importances_

In [ ]:
feature_names = model_cbr.best_estimator_.named_steps["preprocessing"].get_feature_names_out()

In [ ]:
data_feature_importance = pd.DataFrame({'feature_names': feature_names, 'feature_importance': feature_importance})

with pd.option_context('display.max_rows', None,):
    print(data_feature_importance.sort_values(by="feature_importance", ascending=False))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()

fig.suptitle('Важность признаков для модели',
             fontsize=11, fontweight='bold', y=0.95)


data_feature_importance.sort_values(
    by=['feature_importance'],
    ascending=True).plot.barh(x='feature_names', y='feature_importance', ax=ax, legend=False);

ax.set(xlabel='Значение важности', ylabel='Признаки')

plt.show();

In [ ]:
data.info()